In [20]:
import gmsh
import random
import math

def create_random_spheres_in_cube(num_spheres, cube_size, sphere_radius):
    gmsh.initialize()
    gmsh.model.add("random_spheres")

    # Crear el cubo
    gmsh.model.occ.addBox(0, 0, 0, cube_size, cube_size, cube_size)

    # IDs de los volúmenes de las esferas
    sphere_volumes = []

    # Crear esferas en posiciones aleatorias
    for _ in range(num_spheres):
        x = random.uniform(sphere_radius, cube_size - sphere_radius)
        y = random.uniform(sphere_radius, cube_size - sphere_radius)
        z = random.uniform(sphere_radius, cube_size - sphere_radius)
        sphere = gmsh.model.occ.addSphere(x, y, z, sphere_radius)
        sphere_volumes.append(sphere)

    # Fusionar las esferas en un solo volumen
    gmsh.model.occ.synchronize()
    if len(sphere_volumes) > 1:
        _, combined_volume = gmsh.model.occ.fuse([(3, sphere_volumes[0])], [(3, vol) for vol in sphere_volumes[1:]])

    gmsh.model.occ.synchronize()
    gmsh.write("random_spheres.stp")

    # Crear la malla
    # mesh size 
    gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.01)
    gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.5)
    gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 20)

    gmsh.model.mesh.generate(3)
    gmsh.write("random_spheres.inp")
    gmsh.fltk.run()



    gmsh.finalize()

# Parámetros del modelo
num_spheres = 100  # Número de esferas
cube_size = 5   # Tamaño del cubo
sphere_radius = 0.5  # Radio de las esferas

create_random_spheres_in_cube(num_spheres, cube_size, sphere_radius)


In [4]:
import gmsh
import random
import math

def check_distance_ok(x, y, z, spheres, min_dist):
    for sx, sy, sz in spheres:
        if math.sqrt((x - sx)**2 + (y - sy)**2 + (z - sz)**2) < min_dist:
            return False
    return True

def create_random_spheres_in_cube(num_spheres, cube_size, sphere_radius, threshold):
    gmsh.initialize()
    gmsh.model.add("random_spheres_adjusted")

    # Crear el cubo
    gmsh.model.occ.addBox(0, 0, 0, cube_size, cube_size, cube_size)

    # Lista para almacenar las posiciones de las esferas
    spheres = []
    sphere_volumes = []

    # Generar esferas en posiciones aleatorias respetando la distancia mínima
    attempts = 0
    while len(spheres) < num_spheres and attempts < 5000:
        x = random.uniform(sphere_radius, cube_size - sphere_radius)
        y = random.uniform(sphere_radius, cube_size - sphere_radius)
        z = random.uniform(sphere_radius, cube_size - sphere_radius)
        if check_distance_ok(x, y, z, spheres, 2 * sphere_radius + threshold):
            sphere = gmsh.model.occ.addSphere(x, y, z, sphere_radius)
            spheres.append((x, y, z))
            sphere_volumes.append(sphere)

        if attempts % 100 == 0:
            print("Attempt", attempts, "Number of spheres:", len(spheres))
        attempts += 1

    # Fusionar las esferas en un solo volumen
    gmsh.model.occ.synchronize()
    if len(sphere_volumes) > 1:
        _, combined_volume = gmsh.model.occ.fuse([(3, sphere_volumes[0])], [(3, vol) for vol in sphere_volumes[1:]])

    gmsh.model.occ.synchronize()
    gmsh.write("random_spheres_adjusted.stp")
    # mesh 
    #curve 
    gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.01)
    gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.5)
    gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 20)
    gmsh.model.mesh.generate(3)
    gmsh.write("random_spheres_adjusted.inp")
    gmsh.fltk.run()

    gmsh.finalize()

# Parámetros del modelo
num_spheres = 50   # Número de esferas
cube_size = 4    # Tamaño del cubo
sphere_radius = 0.5  # Radio de las esferas
threshold = -0.25   # Distancia mínima adicional entre las esferas

create_random_spheres_in_cube(num_spheres, cube_size, sphere_radius, threshold)


Attempt 0 Number of spheres: 1
Attempt 100 Number of spheres: 32
Attempt 200 Number of spheres: 39
Attempt 300 Number of spheres: 43
Attempt 400 Number of spheres: 45
Attempt 500 Number of spheres: 47
Attempt 600 Number of spheres: 47
Attempt 700 Number of spheres: 47
